# RAG 시스템 평가 (RAG Evaluation)

이번 노트북에서는 **RAG 시스템을 평가하는 방법**을 학습합니다.

## 학습 목표
RAG 시스템의 성능을 정량적으로 측정하고 개선점을 찾는 방법을 익힙니다.

## RAG 평가가 중요한 이유

RAG 시스템은 두 가지 핵심 컴포넌트로 구성됩니다:
1. **Retrieval (검색)**: 관련 문서를 얼마나 잘 찾는가?
2. **Generation (생성)**: 찾은 문서를 바탕으로 얼마나 좋은 답변을 생성하는가?

각 컴포넌트를 개별적으로 평가해야 문제점을 정확히 파악할 수 있습니다.

## 오늘 배울 내용

### 1. 검색 평가 지표 (Retrieval Metrics)
- **MRR (Mean Reciprocal Rank)**: 정답이 몇 번째에 등장하는가?
- **nDCG (Normalized Discounted Cumulative Gain)**: 관련 문서들의 순위 품질
- **Keyword Coverage**: 필수 키워드가 검색 결과에 포함되는가?

### 2. 답변 평가 방법 (Answer Evaluation)
- **LLM-as-a-Judge**: LLM을 활용한 답변 품질 평가
- 평가 기준: Accuracy, Completeness, Relevance

## 환경 설정

필요한 라이브러리를 임포트합니다.

In [ ]:
import os
import json
import math
import glob
from pathlib import Path
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from litellm import completion

# LangChain 관련
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.documents import Document

load_dotenv(override=True)

# 설정
MODEL = "gpt-4.1-nano"
DB_NAME = "vector_db"
KNOWLEDGE_BASE = "example/knowledge-base"
TEST_FILE = "example/rag_evaluation_basic/tests.jsonl"
RETRIEVAL_K = 10

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

## 1. 테스트 데이터 구조 정의

RAG 평가를 위해서는 **테스트 데이터셋**이 필요합니다.

각 테스트 케이스는 다음 정보를 포함합니다:
- `question`: RAG 시스템에 던질 질문
- `keywords`: 검색 결과에 반드시 포함되어야 하는 키워드들
- `reference_answer`: 정답 (Ground Truth)
- `category`: 질문 유형 (direct_fact, temporal, spanning 등)

In [ ]:
class TestQuestion(BaseModel):
    """RAG 평가용 테스트 질문 모델"""
    
    question: str = Field(description="RAG 시스템에 던질 질문")
    keywords: list[str] = Field(description="검색 결과에 포함되어야 하는 키워드들")
    reference_answer: str = Field(description="정답 (Ground Truth)")
    category: str = Field(description="질문 카테고리 (direct_fact, temporal, spanning 등)")


def load_tests() -> list[TestQuestion]:
    """JSONL 파일에서 테스트 케이스 로드"""
    tests = []
    with open(TEST_FILE, "r", encoding="utf-8") as f:
        for line in f:
            data = json.loads(line)
            tests.append(TestQuestion(**data))
    return tests

In [ ]:
# 테스트 데이터 로드 및 확인
tests = load_tests()
print(f"총 {len(tests)}개의 테스트 케이스 로드됨")

# 첫 번째 테스트 케이스 확인
test = tests[0]
print(f"\n[테스트 예시]")
print(f"Question: {test.question}")
print(f"Keywords: {test.keywords}")
print(f"Reference Answer: {test.reference_answer}")
print(f"Category: {test.category}")

### 테스트 카테고리 설명

| 카테고리 | 설명 | 예시 |
|---------|------|------|
| `direct_fact` | 단일 문서에서 직접 찾을 수 있는 사실 | "CEO의 연봉은?" |
| `temporal` | 시간/날짜 관련 질문 | "언제 설립되었나?" |
| `spanning` | 여러 문서를 종합해야 하는 질문 | "IIOTY 수상자가 담당하는 제품은?" |
| `numerical` | 숫자/통계 관련 질문 | "직원 수는?" |
| `comparative` | 비교/분석 질문 | "가장 비싼 요금제는?" |
| `holistic` | 전체적인 이해가 필요한 질문 | "총 계약 건수는?" |

In [ ]:
# 카테고리별 테스트 분포 확인
from collections import Counter

category_counts = Counter(test.category for test in tests)
print("카테고리별 테스트 분포:")
for category, count in sorted(category_counts.items()):
    print(f"  {category}: {count}개")

## 2. 기본 RAG 시스템 구축

평가 대상이 될 RAG 시스템을 LangChain으로 구축합니다.

### 2.1 데이터 수집 (Ingest)

In [ ]:
def fetch_documents():
    """Knowledge Base에서 문서 로드"""
    folders = glob.glob(str(Path(KNOWLEDGE_BASE) / "*"))
    documents = []
    
    for folder in folders:
        doc_type = os.path.basename(folder)
        loader = DirectoryLoader(
            folder, 
            glob="**/*.md", 
            loader_cls=TextLoader, 
            loader_kwargs={"encoding": "utf-8"}
        )
        folder_docs = loader.load()
        
        for doc in folder_docs:
            doc.metadata["doc_type"] = doc_type
            documents.append(doc)
    
    return documents


def create_chunks(documents):
    """문서를 청크로 분할"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,      # 청크 크기
        chunk_overlap=200    # 오버랩 크기
    )
    chunks = text_splitter.split_documents(documents)
    return chunks


def create_vectorstore(chunks):
    """벡터 스토어 생성"""
    if os.path.exists(DB_NAME):
        Chroma(persist_directory=DB_NAME, embedding_function=embeddings).delete_collection()
    
    vectorstore = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings,
        persist_directory=DB_NAME
    )
    
    print(f"벡터스토어 생성 완료: {vectorstore._collection.count()}개 문서")
    return vectorstore

In [ ]:
# 데이터 수집 실행 (이미 생성되어 있다면 스킵 가능)
# documents = fetch_documents()
# print(f"로드된 문서: {len(documents)}개")

# chunks = create_chunks(documents)
# print(f"생성된 청크: {len(chunks)}개")

# vectorstore = create_vectorstore(chunks)

### 2.2 질문 응답 (Answer)

벡터 스토어에서 관련 문서를 검색하고 LLM으로 답변을 생성합니다.

In [ ]:
# 기존 벡터스토어 로드
vectorstore = Chroma(
    persist_directory="example/rag_evaluation_basic/vector_db",
    embedding_function=embeddings
)
retriever = vectorstore.as_retriever(search_kwargs={"k": RETRIEVAL_K})
llm = ChatOpenAI(model=MODEL, temperature=0)

SYSTEM_PROMPT = """
You are a knowledgeable, friendly assistant representing the company Insurellm.
You are chatting with a user about Insurellm.
If relevant, use the given context to answer any question.
If you don't know the answer, say so.
Context:
{context}
"""


def fetch_context(question: str) -> list[Document]:
    """질문과 관련된 문서 검색"""
    return retriever.invoke(question)


def answer_question(question: str, history: list[dict] = []) -> tuple[str, list[Document]]:
    """
    RAG 방식으로 질문에 답변
    
    Returns:
        tuple: (답변 문자열, 검색된 문서 리스트)
    """
    # 1. 관련 문서 검색
    context_docs = fetch_context(question)
    context_text = "\n\n".join(doc.page_content for doc in context_docs)
    
    # 2. 프롬프트 구성
    system_message = SystemMessage(content=SYSTEM_PROMPT.format(context=context_text))
    messages = [system_message, HumanMessage(content=question)]
    
    # 3. LLM 호출
    response = llm.invoke(messages)
    
    return response.content, context_docs

In [ ]:
# RAG 시스템 테스트
test_question = "Who won the IIOTY award in 2023?"
answer, docs = answer_question(test_question)

print(f"질문: {test_question}")
print(f"\n답변: {answer}")
print(f"\n검색된 문서 수: {len(docs)}")

## 3. 검색 평가 (Retrieval Evaluation)

검색 품질을 측정하는 핵심 지표들을 구현합니다.

### 3.1 MRR (Mean Reciprocal Rank)

**정의**: 정답 문서가 검색 결과에서 몇 번째에 등장하는지 측정

**계산 방법**:
- 정답이 1번째: 1/1 = 1.0
- 정답이 2번째: 1/2 = 0.5
- 정답이 3번째: 1/3 = 0.33
- 정답이 없음: 0

**해석**:
- MRR = 1.0: 항상 첫 번째에 정답 (완벽)
- MRR ≥ 0.9: 우수
- MRR ≥ 0.75: 양호
- MRR < 0.75: 개선 필요

In [ ]:
def calculate_mrr(keyword: str, retrieved_docs: list) -> float:
    """
    단일 키워드에 대한 MRR 계산
    
    Args:
        keyword: 검색 결과에서 찾아야 하는 키워드
        retrieved_docs: 검색된 문서 리스트
        
    Returns:
        Reciprocal Rank (정답 순위의 역수)
    """
    keyword_lower = keyword.lower()
    
    for rank, doc in enumerate(retrieved_docs, start=1):
        if keyword_lower in doc.page_content.lower():
            return 1.0 / rank  # 첫 번째 발견 위치의 역수
    
    return 0.0  # 키워드를 찾지 못함

In [ ]:
# MRR 예시
test = tests[0]  # "Who won the IIOTY award in 2023?"
docs = fetch_context(test.question)

print(f"질문: {test.question}")
print(f"찾아야 할 키워드: {test.keywords}")
print()

for keyword in test.keywords:
    mrr = calculate_mrr(keyword, docs)
    print(f"  '{keyword}' MRR: {mrr:.4f}")

### 3.2 nDCG (Normalized Discounted Cumulative Gain)

**정의**: 검색 결과의 전체적인 순위 품질을 측정

**MRR과의 차이**:
- MRR: 첫 번째 정답 위치만 고려
- nDCG: 모든 관련 문서의 위치를 고려

**계산 방법**:
1. DCG = Σ(relevance_i / log2(rank_i + 1))
2. IDCG = 이상적인 순서일 때의 DCG
3. nDCG = DCG / IDCG

**해석**:
- nDCG = 1.0: 완벽한 순위 (관련 문서가 모두 상위에 위치)
- nDCG ≥ 0.9: 우수
- nDCG < 0.75: 개선 필요

In [ ]:
def calculate_dcg(relevances: list[int], k: int) -> float:
    """Discounted Cumulative Gain 계산"""
    dcg = 0.0
    for i in range(min(k, len(relevances))):
        # 순위가 낮을수록 (뒤에 있을수록) 가치가 할인됨
        dcg += relevances[i] / math.log2(i + 2)  # i+2: rank 1부터 시작
    return dcg


def calculate_ndcg(keyword: str, retrieved_docs: list, k: int = 10) -> float:
    """
    단일 키워드에 대한 nDCG 계산 (Binary Relevance)
    
    Args:
        keyword: 찾아야 하는 키워드
        retrieved_docs: 검색된 문서 리스트
        k: 상위 k개 문서만 고려
        
    Returns:
        Normalized DCG 값 (0~1)
    """
    keyword_lower = keyword.lower()
    
    # Binary relevance: 키워드 포함 시 1, 아니면 0
    relevances = [
        1 if keyword_lower in doc.page_content.lower() else 0
        for doc in retrieved_docs[:k]
    ]
    
    # 실제 DCG
    dcg = calculate_dcg(relevances, k)
    
    # 이상적인 DCG (관련 문서가 모두 앞에 있을 때)
    ideal_relevances = sorted(relevances, reverse=True)
    idcg = calculate_dcg(ideal_relevances, k)
    
    return dcg / idcg if idcg > 0 else 0.0

In [ ]:
# nDCG 예시
test = tests[0]
docs = fetch_context(test.question)

print(f"질문: {test.question}")
print()

for keyword in test.keywords:
    ndcg = calculate_ndcg(keyword, docs)
    print(f"  '{keyword}' nDCG: {ndcg:.4f}")

### 3.3 검색 평가 통합

In [ ]:
class RetrievalEval(BaseModel):
    """검색 평가 결과 모델"""
    
    mrr: float = Field(description="Mean Reciprocal Rank (평균)")
    ndcg: float = Field(description="Normalized DCG (평균)")
    keywords_found: int = Field(description="찾은 키워드 수")
    total_keywords: int = Field(description="전체 키워드 수")
    keyword_coverage: float = Field(description="키워드 커버리지 (%)")


def evaluate_retrieval(test: TestQuestion, k: int = 10) -> RetrievalEval:
    """
    단일 테스트에 대한 검색 평가 수행
    
    Args:
        test: 테스트 케이스
        k: 상위 k개 문서 검색
        
    Returns:
        RetrievalEval 결과 객체
    """
    # 문서 검색
    retrieved_docs = fetch_context(test.question)
    
    # 각 키워드에 대해 MRR, nDCG 계산
    mrr_scores = [calculate_mrr(kw, retrieved_docs) for kw in test.keywords]
    ndcg_scores = [calculate_ndcg(kw, retrieved_docs, k) for kw in test.keywords]
    
    # 평균 계산
    avg_mrr = sum(mrr_scores) / len(mrr_scores) if mrr_scores else 0.0
    avg_ndcg = sum(ndcg_scores) / len(ndcg_scores) if ndcg_scores else 0.0
    
    # 키워드 커버리지
    keywords_found = sum(1 for score in mrr_scores if score > 0)
    total_keywords = len(test.keywords)
    keyword_coverage = (keywords_found / total_keywords * 100) if total_keywords > 0 else 0.0
    
    return RetrievalEval(
        mrr=avg_mrr,
        ndcg=avg_ndcg,
        keywords_found=keywords_found,
        total_keywords=total_keywords,
        keyword_coverage=keyword_coverage
    )

In [ ]:
# 검색 평가 실행 예시
test = tests[0]
result = evaluate_retrieval(test)

print(f"질문: {test.question}")
print(f"\n[검색 평가 결과]")
print(f"  MRR: {result.mrr:.4f}")
print(f"  nDCG: {result.ndcg:.4f}")
print(f"  Keywords Found: {result.keywords_found}/{result.total_keywords}")
print(f"  Keyword Coverage: {result.keyword_coverage:.1f}%")

## 4. 답변 평가 (Answer Evaluation)

### LLM-as-a-Judge

LLM을 활용하여 생성된 답변의 품질을 평가하는 방법입니다.

**평가 기준:**

| 기준 | 설명 | 점수 기준 |
|------|------|----------|
| **Accuracy** | 사실적 정확성 | 1점(오답) ~ 5점(완벽) |
| **Completeness** | 답변의 완전성 | 1점(불완전) ~ 5점(완전) |
| **Relevance** | 질문과의 관련성 | 1점(무관) ~ 5점(직접 관련) |

**왜 LLM-as-a-Judge인가?**
- 자연어 답변은 정확히 일치하지 않아도 정답일 수 있음
- 의미적 동등성을 사람처럼 판단 가능
- 대규모 평가를 자동화할 수 있음

In [ ]:
class AnswerEval(BaseModel):
    """LLM-as-a-Judge 답변 평가 결과"""
    
    feedback: str = Field(
        description="답변 품질에 대한 피드백"
    )
    accuracy: float = Field(
        description="사실적 정확도 (1~5점). 오답은 반드시 1점"
    )
    completeness: float = Field(
        description="답변의 완전성 (1~5점). 모든 정보 포함 시 5점"
    )
    relevance: float = Field(
        description="질문과의 관련성 (1~5점). 직접 답변 시 5점"
    )

In [ ]:
def evaluate_answer(test: TestQuestion) -> tuple[AnswerEval, str, list]:
    """
    LLM-as-a-Judge 방식으로 답변 품질 평가
    
    Args:
        test: 테스트 케이스 (질문 + 정답)
        
    Returns:
        tuple: (평가 결과, 생성된 답변, 검색된 문서들)
    """
    # RAG로 답변 생성
    generated_answer, retrieved_docs = answer_question(test.question)
    
    # LLM Judge 프롬프트
    judge_messages = [
        {
            "role": "system",
            "content": """You are an expert evaluator assessing the quality of answers.
Evaluate the generated answer by comparing it to the reference answer.
Only give 5/5 scores for perfect answers."""
        },
        {
            "role": "user",
            "content": f"""Question:
{test.question}

Generated Answer:
{generated_answer}

Reference Answer:
{test.reference_answer}

Please evaluate the generated answer on three dimensions:
1. Accuracy: How factually correct is it compared to the reference answer?
2. Completeness: How thoroughly does it address all aspects of the question?
3. Relevance: How well does it directly answer the specific question asked?

Provide detailed feedback and scores from 1 (very poor) to 5 (ideal) for each dimension.
If the answer is wrong, then the accuracy score must be 1."""
        }
    ]
    
    # LLM Judge 호출 (Structured Output)
    judge_response = completion(
        model=MODEL, 
        messages=judge_messages, 
        response_format=AnswerEval
    )
    
    answer_eval = AnswerEval.model_validate_json(
        judge_response.choices[0].message.content
    )
    
    return answer_eval, generated_answer, retrieved_docs

In [ ]:
# 답변 평가 실행 예시
test = tests[0]
eval_result, generated, docs = evaluate_answer(test)

print(f"질문: {test.question}")
print(f"\n정답: {test.reference_answer}")
print(f"\n생성된 답변: {generated}")
print(f"\n[답변 평가 결과]")
print(f"  Feedback: {eval_result.feedback}")
print(f"  Accuracy: {eval_result.accuracy}/5")
print(f"  Completeness: {eval_result.completeness}/5")
print(f"  Relevance: {eval_result.relevance}/5")

## 5. 전체 평가 실행

모든 테스트 케이스에 대해 평가를 실행하고 결과를 집계합니다.

In [ ]:
from tqdm import tqdm

def evaluate_all_retrieval(tests: list[TestQuestion]) -> dict:
    """
    전체 테스트에 대한 검색 평가 실행
    
    Returns:
        집계된 평가 결과 딕셔너리
    """
    total_mrr = 0.0
    total_ndcg = 0.0
    total_coverage = 0.0
    category_results = {}
    
    for test in tqdm(tests, desc="검색 평가 중"):
        result = evaluate_retrieval(test)
        
        total_mrr += result.mrr
        total_ndcg += result.ndcg
        total_coverage += result.keyword_coverage
        
        # 카테고리별 집계
        if test.category not in category_results:
            category_results[test.category] = []
        category_results[test.category].append(result.mrr)
    
    n = len(tests)
    return {
        "avg_mrr": total_mrr / n,
        "avg_ndcg": total_ndcg / n,
        "avg_coverage": total_coverage / n,
        "category_mrr": {
            cat: sum(scores) / len(scores) 
            for cat, scores in category_results.items()
        }
    }

In [ ]:
# 전체 검색 평가 실행 (시간이 걸릴 수 있음)
# 샘플로 처음 10개만 평가
sample_tests = tests[:10]
retrieval_results = evaluate_all_retrieval(sample_tests)

print("\n[전체 검색 평가 결과]")
print(f"  평균 MRR: {retrieval_results['avg_mrr']:.4f}")
print(f"  평균 nDCG: {retrieval_results['avg_ndcg']:.4f}")
print(f"  평균 Keyword Coverage: {retrieval_results['avg_coverage']:.1f}%")

print("\n[카테고리별 MRR]")
for cat, mrr in retrieval_results['category_mrr'].items():
    print(f"  {cat}: {mrr:.4f}")

## 6. 결과 해석 및 개선 방향

### 평가 결과 해석 가이드

| 지표 | 우수 | 양호 | 개선 필요 |
|------|------|------|----------|
| MRR | ≥ 0.9 | ≥ 0.75 | < 0.75 |
| nDCG | ≥ 0.9 | ≥ 0.75 | < 0.75 |
| Coverage | ≥ 90% | ≥ 75% | < 75% |
| Accuracy | ≥ 4.5 | ≥ 4.0 | < 4.0 |
| Completeness | ≥ 4.5 | ≥ 4.0 | < 4.0 |
| Relevance | ≥ 4.5 | ≥ 4.0 | < 4.0 |

### 개선 방향

**검색 품질이 낮을 때:**
- 청크 크기 조정
- 임베딩 모델 변경 (예: text-embedding-3-large)
- Reranking 적용
- Query Rewriting 적용

**답변 품질이 낮을 때:**
- 시스템 프롬프트 개선
- 더 많은 컨텍스트 제공 (K 값 증가)
- LLM 모델 업그레이드

## 정리

이번 노트북에서 학습한 내용:

### 1. RAG 평가의 필요성
- 검색(Retrieval)과 생성(Generation)을 개별 평가
- 문제점을 정확히 파악하여 개선 방향 도출

### 2. 검색 평가 지표
- **MRR**: 첫 번째 정답 위치의 역수
- **nDCG**: 전체 순위 품질
- **Keyword Coverage**: 필수 키워드 포함률

### 3. 답변 평가 방법
- **LLM-as-a-Judge**: LLM을 활용한 자동 평가
- 평가 기준: Accuracy, Completeness, Relevance

### 4. 다음 단계
- 고급 RAG 기법 적용 (Reranking, Query Rewriting)
- 평가 결과 비교를 통한 최적 설정 탐색
- Gradio UI를 통한 평가 대시보드 구축